출력 파서: CommaSeparatedListOutputParser

문제

사용자가 관심있는 분야(예: "음식", "스포츠", "영화" 등)를 입력하면, 해당 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된 리스트로 출력하는 프로그램을 작성하세요.

요구사항:

CommaSeparatedListOutputParser 사용
사용자 입력을 받아 동적으로 처리
한국 관련 내용으로 추천
예시 입력: "음식"
예시 출력: 
['명동 칼국수', '부산 돼지국밥', '전주 비빔밥', '춘천 닭갈비', '제주 흑돼지']


In [9]:
import os
from dotenv import load_dotenv
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv
from pprint import pprint

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="{subject}과 관련된 한국의 유명한 장소나 활동 5가지를 보내시오.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)
pprint(prompt.partial_variables)

# OpenAI 모델 설정
#model = ChatOpenAI(temperature=0)
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0
)

# 프롬프트, 모델, 출력 파서를 연결하여 체인 생성
chain = prompt | model | output_parser

subject = "음식"
# "AI 관련 기술"에 대한 체인 호출 실행
result = chain.invoke({"subject": subject})

# 쉼표로 구분된 리스트 출력
print(f"{subject} 관련 명소 및 활동 : ")
print(result)


{'format_instructions': 'Your response should be a list of comma separated '
                        'values, eg: `foo, bar, baz` or `foo,bar,baz`'}
음식 관련 명소 및 활동 : 
['서울의 N서울타워', '경주의 황남빵', '부산의 자갈치시장', '전주의 비빔밥', '남이섬의 닭갈비']


문제 2-2 : 영화 리뷰 감정 분석기

출력 파서: EnumOutputParser

문제

영화 리뷰 텍스트를 입력받아 감정을 "긍정", "부정", "보통" 중 하나로 분류하는 시스템을 만드세요.

요구사항:

EnumOutputParser와 Enum 클래스 사용
여러 개의 테스트 리뷰로 검증
감정 분류 결과를 깔끔하게 출력
테스트 리뷰 예시:
"이 영화 정말 재미없어요. 시간 낭비였습니다."
"배우들의 연기가 훌륭하고 스토리도 감동적이었어요!"
"그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint
import os

# 환경변수 로드
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# 감정 클래스 정의 (Enum)
class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "중립"

# EnumOutputParser 초기화
parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

print("감정 분류 출력 형식:")
print(format_instructions)

# 프롬프트 템플릿
template = """
당신은 영화 리뷰 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)   

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0  # 일관성을 위해 0으로 설정
)

# OutputFixingParser로 안정성 향상
fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

print("모델 및 파서 설정 완료")

test_reviews = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
]

print(f"테스트할 텍스트 {len(test_reviews)}개 준비 완료")

# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

# 실제 감정 분석 실행 (API 키 필요)
def run_sentiment_analysis():
    """실제 감정 분석 실행"""
    print("=== 실제 감정 분석 결과 ===")
    
    success_count = 0
    total_count = len(test_reviews)
    
    for i, text in enumerate(test_reviews, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
    print(f"\n=== 결과 요약 ===")
    print(f"성공: {success_count}/{total_count} ({success_count/total_count*100:.1f}%)")
    print(f"실패: {total_count-success_count}/{total_count}")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")

감정 분류 출력 형식:
Select one of the following options: 긍정, 부정, 중립
모델 및 파서 설정 완료
테스트할 텍스트 0개 준비 완료
=== 실제 감정 분석 결과 ===

1. 텍스트: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 

2. 텍스트: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 

3. 텍스트: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 중립 

=== 결과 요약 ===
성공: 3/3 (100.0%)
실패: 0/3


문제 2-3: 학생 정보 구조화 시스템

출력 파서: PydanticOutputParser

문제

학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태로 추출하는 시스템을 만드세요.

요구사항:

PydanticOutputParser와 BaseModel 사용
각 필드에 적절한 타입과 설명 추가
자유 형식의 텍스트에서 정보 추출

예시 입력:

 "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

예시 출력 구조:

{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": ["게임하기", "영화보기", "코딩"],
    "goal": "훌륭한 개발자가 되는 것"
}


In [3]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

import json

# Pydantic 모델 정의
class StudentInfo(BaseModel):
    name: str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미 리스트")
    goal: str = Field(description="학생의 목표")

# 출력 파서
parser = PydanticOutputParser(pydantic_object=StudentInfo)

# 프롬프트 템플릿 설정
template = """
다음은 학생의 자유 형식 자기소개입니다. 이 자기소개에서 아래 항목을 추출하여 JSON 형식으로 출력하세요:

- name: 학생의 이름
- age: 학생의 나이 (숫자만)
- major: 학생의 전공
- hobbies: 쉼표로 구분된 취미들을 리스트로 추출
- goal: 학생의 목표

출력은 반드시 아래 형식을 따르세요:

{format_instructions}

자기소개:
{input}
"""

prompt = ChatPromptTemplate.from_template(template)

prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

# LLM 모델 설정
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 구성 및 실행
chain = prompt | model | parser
intro_text = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."
output = chain.invoke({"input": intro_text})

# 결과 출력
print(json.dumps(output.model_dump(), ensure_ascii=False, indent=4))

{
    "name": "김민수",
    "age": 22,
    "major": "컴퓨터공학",
    "hobbies": [
        "게임하기",
        "영화보기",
        "코딩"
    ],
    "goal": "훌륭한 개발자가 되는 것"
}


문제 2-4 : 여행 계획 분석기

출력 파서: StructuredOutputParser

문제

여행 후기나 계획 텍스트를 입력받아 여행지, 기간, 예산, 추천도(1-5점), 주요 활동 리스트를 구조화된 형태로 추출하는 시스템을 만드세요.

요구사항:

StructuredOutputParser와 ResponseSchema 사용
5개의 필드를 정의하여 정보 추출
자연어 텍스트에서 핵심 정보 파싱

예시 입력:

 "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."

예시 출력 구조:

{
    "destination": "부산",
    "duration": "2박 3일",
    "budget": "30만원",
    "rating": "4",
    "activities": ["해운대 바다구경", "자갈치시장 회 먹기", "감천문화마을 구경"]
}


In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
import json
# 출력 구조 정의
response_schemas = [
    ResponseSchema(name="destination", description="여행지 (예: 부산)"),
    ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일)"),
    ResponseSchema(name="budget", description="여행 예산 (예: 30만원)"),
    ResponseSchema(name="rating", description="추천도. 1~5 중 숫자"),
    ResponseSchema(name="activities", description="주요 활동 리스트. 반드시 리스트 형식으로 출력 (예: ['해운대 바다구경', '자갈치시장 회 먹기'])")
]

# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿
template = """
다음은 여행 후기 또는 계획입니다. 아래 항목에 따라 정보를 추출하여 JSON 형식으로 출력하세요:

- destination: 여행지
- duration: 여행 기간
- budget: 예산
- rating: 추천도 (1~5점)
- activities: 주요 활동 리스트 (반드시 Python 리스트 형식으로 출력하세요)

출력 형식은 다음 지침을 따르세요:
{format_instructions}

여행 텍스트:
{input}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

# 모델 초기화
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.7
)

# 체인 실행
chain = prompt | model | parser
travel_text = (
    "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
)
output = chain.invoke({"input": travel_text})

# 결과 출력
print(json.dumps(output, ensure_ascii=False, indent=4))

{
    "destination": "부산",
    "duration": "2박3일",
    "budget": "30만원",
    "rating": "4",
    "activities": [
        "해운대 바다구경",
        "자갈치시장 회 먹기",
        "감천문화마을 구경"
    ]
}
